# Interactive combined fit of 4 correlators

Interactively peform a combined fit of 4pt function with 'l' values 0,2,4,6


Dec 27, 2022

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ipywidgets import *
import glob 
import sys
import pickle 
import scipy.special as sc
# from scipy.special import logsumexp as sc.logsumexp

## Import fitting modules
import gvar as gv
import lsqfit


In [2]:
%matplotlib widget

## Modules

In [3]:
def f_scale_4pt(row,s):
    a1=row['coeff']/(dict_global[str(s)]['2pt']**2) # divide by phi_2pt ^ 2 
    return a1

def f_get_data_df(fname):

    arr=np.loadtxt(fname)

    ## Extract values from file name
    s=int(fname.split('/')[-1].split('_')[2].split('k')[-1].split('t')[0])
    Lt=int(fname.split('/')[-1].split('_')[2].split('k')[-1].split('t')[-1])
    
    print(s,Lt)
    Nsites=10*(s**2)+2
    # c=dict_global[str(s)]['c']

    ### Gather data from file
    cols=['l','t','coeff']
    df_data=pd.DataFrame(arr,columns=cols)

    ### Process data
    # df_data.apply(lambda row: f_scale_4pt(row,s),axis=1)
    df_data['coeff']=df_data.apply(lambda row: f_scale_4pt(row,s),axis=1)
    df_data.loc[df_data.l==0,['coeff']]-=1 ## subtract out the 1 for l=0
    
    # df_data['coeff']=np.array([gv.gvar(i,i*0.01) for i in df_data.coeff.values])
    df_data['coeff']=np.array([gv.gvar(i,max(i*1e-4,1e-13)) for i in df_data.coeff.values]) # Ensure error doesn't go below machine precision
    

    return df_data,Lt


### Class definition

In [4]:
class corr:
    ''' Class to store correlators C(t) for specific s'''
    def __init__(self,df1,df2,df3,df4,c,s,Lt):
        self.df1=df1
        self.df2=df2
        self.df3=df3
        self.df4=df4
        
        self.c=c
        self.s=s
        self.Lt=Lt
        
        self.x_full=None
        self.y_full=None
        
        self.fit=None
        
    def f_perform_fit_exp(self,f_make_pars,func,fit_range1,fit_range2,fit_range3, fit_range4, plt_range,num_exp=3,verbose=0,use_prior=False,plot=True):
              
        ## Define x and y for object for full plot range (Used only for fit)
        self.x_full={'t1':self.df1.t.values[min(plt_range):max(plt_range)+1],
                     't2':self.df2.t.values[min(plt_range):max(plt_range)+1],
                     't3':self.df3.t.values[min(plt_range):max(plt_range)+1],
                     't4':self.df4.t.values[min(plt_range):max(plt_range)+1],
                     'nterms':num_exp,'c':self.c,'Lt':self.Lt}
        
        self.y_full={'y1': self.df1.coeff.values[min(plt_range):max(plt_range)+1],
                     'y2': self.df2.coeff.values[min(plt_range):max(plt_range)+1],
                     'y3': self.df3.coeff.values[min(plt_range):max(plt_range)+1], 
                     'y4': self.df4.coeff.values[min(plt_range):max(plt_range)+1] }
        
        x=self.x_full.copy();
        x['t1']=x['t1'][min(fit_range1):max(fit_range1)+1]
        x['t2']=x['t2'][min(fit_range2):max(fit_range2)+1]
        x['t3']=x['t3'][min(fit_range3):max(fit_range3)+1]
        x['t4']=x['t4'][min(fit_range4):max(fit_range4)+1]
        
        y1=self.y_full['y1'][min(fit_range1):max(fit_range1)+1]
        y2=self.y_full['y2'][min(fit_range2):max(fit_range2)+1]
        y3=self.y_full['y3'][min(fit_range3):max(fit_range3)+1]
        y4=self.y_full['y4'][min(fit_range4):max(fit_range4)+1]
        
        y={'y1':y1, 'y2':y2, 'y3':y3, 'y4':y4}
                
        ### Create parameters dictionary: if prior gives gvars
        p0=f_make_pars(num_exp,use_prior)
        
#         print(x,y)
        if use_prior:
            self.fit = lsqfit.nonlinear_fit(data=(x,y), prior=p0, fcn=func)
        else: 
            self.fit = lsqfit.nonlinear_fit(data=(x,y), p0=p0, fcn=func)
        
        ### Print fit details before plot
        if verbose==2: print(self.fit.format(maxline=True))
        elif verbose: print(self.fit.format(maxline=verbose))
        
        if plot: self.f_fit_plot(min(plt_range),max(plt_range),error_band=True,semilog=True)
        
        # Print description at the end
        if verbose:
            for k in self.fit.p.keys():
                if not use_prior:
                    print(k,'\tInit',self.fit.p0[k],'\t---Final',self.fit.p[k])
                else : 
                    print(k,'\tInit',self.fit.prior[k],'\t---Final',self.fit.p[k])    

            print("\nchi-sqr",self.fit.chi2/self.fit.dof)

        
    def f_fit_plot(self,start=0,end=None,error_band=True,semilog=True):
        '''
        Function for plotting data with the fit lines and error bands.
        For correlators, using a semi-log plot.
        error_band=True, plots a band in the full region
        '''
        
        
        ## Create y_predictions for fit function
        x=self.x_full; y=self.y_full
        curve_t={}
        curve_x=self.x_full.copy()
        
        num_corrs=4
        for i in range(1,num_corrs+1):
            key_t='t%s'%(i)            
            curve_t[key_t]=np.linspace(min(x[key_t]),max(x[key_t]),500)
            curve_x[key_t]=curve_t[key_t]; ## Modify curve_x['ts'] to the range required
        
        
        curve_y=self.fit.fcn(curve_x,self.fit.p)
        
        obs_fit=gv.mean(curve_y)
        err_fit=gv.sdev(curve_y)
        sigma=2.0 # Width for band

        #### Plots
        fig=plt.figure(figsize=(12,4))
        plt.title("Plot")

        plt.xticks([])
        plt.yticks([])
        
        for i in range(1,num_corrs+1):
            key_t='t%s'%(i)
            key_y='y%s'%(i)
            
            fig.add_subplot(2,2,i)
        
            # Plot all data points
            x=self.x_full; y=self.y_full
            plt.errorbar(x[key_t],y=gv.mean(y[key_y]),yerr=gv.sdev(y[key_y]),linestyle='None',color='black',marker='*',markersize=4)

            # Plot the best fit line
            plt.plot(curve_t[key_t],obs_fit[key_y],color='blue')

            if error_band: # Plot an error band around the best fit line
                plt.fill_between(curve_t[key_t],obs_fit[key_y]-sigma*err_fit[key_y],obs_fit[key_y]+sigma*err_fit[key_y],color='yellow')

            ## Plot data points used in fit with different color
            x=self.fit.x; y=self.fit.y
            plt.errorbar(x[key_t],y=gv.mean(y[key_y]),yerr=gv.sdev(y[key_y]),linestyle='None',color='red',marker='H',markersize=5)    

            if semilog: plt.yscale('log')
            plt.ylabel('C_{0}(t)'.format(2*(i-1)))
            plt.xlabel('t')
            
        plt.tight_layout()
        plt.show()

### Fit functions

In [5]:
def f_multi_exp2(x,p):
    '''
    l -> exponents
    0 -> 1, 3, 5, 7
    2 -> 2(wrap), 3, 5, 7
    4 -> 3(SB), 5, 7, 9
    6 -> 1(SB), 3(SB), 6(wrap), 7, 9
    
    Eqn form: 
    
    y1= a11 * E1       + a12 * E3   + a13 * E5      + a14 * E7
    
    y2= a21 * E2       + a22 * E3   + a23 * E5      + a24 * E7
    
    y3= a31 * E3       + a32 * E5   + a33 * E7      + a34 * E9
    
    y4= a41 * E1       + a42 * E3   + a43 * E6      + a44 * E7
    
    E2=E3-E1
    E6=E7-E1
    E4=E5-E1
    
    l=4 might have a 4
    '''
    
    t1=x['t1']
    t2=x['t2']
    t3=x['t3']
    t4=x['t4']
    
    num_exp=x['nterms']
    c=x['c']
    Lt=x['Lt']
    g=1
    
    y3=0
    y2=0
    
    y1=p['const1']
#     y2=p['const2']
#     y3=p['const3']
    y4=p['const4']
    
    if num_exp>0:
        y1+=p['a11']*( np.exp(-p['E1']*g*c*t1) + np.exp(-p['E1']*g*c*(Lt-t1)) )
        y4+=p['a41']*( np.exp(-p['E1']*g*c*t4) + np.exp(-p['E1']*g*c*(Lt-t4)) )
    
    if num_exp>1:
        y1+=p['a12']*( np.exp(-p['E3']*g*c*t1) + np.exp(-p['E3']*g*c*(Lt-t1)) )
        
        y2+=p['a21']*( np.exp(-p['E2']*g*c*t2) + np.exp(-p['E2']*g*c*(Lt-t2)) )
        y2+=p['a22']*( np.exp(-p['E3']*g*c*t2) + np.exp(-p['E3']*g*c*(Lt-t2)) )
        
        y3+=p['a31']*( np.exp(-p['E3']*g*c*t3) + np.exp(-p['E3']*g*c*(Lt-t3)) )
        y4+=p['a42']*( np.exp(-p['E3']*g*c*t4) + np.exp(-p['E3']*g*c*(Lt-t4)) )
    
    if num_exp>2:
        y1+=p['a13']*( np.exp(-p['E5']*g*c*t1) + np.exp(-p['E5']*g*c*(Lt-t1)) )
        y2+=p['a23']*( np.exp(-p['E5']*g*c*t2) + np.exp(-p['E5']*g*c*(Lt-t2)) )
        y3+=p['a32']*( np.exp(-p['E5']*g*c*t3) + np.exp(-p['E5']*g*c*(Lt-t3)) )
#         y4+=p['a43']*( np.exp(-(p['E7']-p['E1'])*g*c*t4) + np.exp(-(p['E7']-p['E1'])*g*c*(Lt-t4)) )

    if num_exp>3:
        y1+=p['a14']*( np.exp(-p['E7']*g*c*t1) + np.exp(-p['E7']*g*c*(Lt-t1)) )
        y2+=p['a24']*( np.exp(-p['E7']*g*c*t2) + np.exp(-p['E7']*g*c*(Lt-t2)) )

        y3+=p['a33']*( np.exp(-p['E7']*g*c*t3) + np.exp(-p['E7']*g*c*(Lt-t3)) )
    
        y4+=p['a43']*( np.exp(-p['E6']*g*c*t4) + np.exp(-p['E6'])*g*c*(Lt-t4) )
        y4+=p['a44']*( np.exp(-p['E7']*g*c*t4) + np.exp(-p['E7']*g*c*(Lt-t4)) )

    if num_exp>4 :
        raise SystemError
        
    return {'y1':y1,'y2':y2, 'y3':y3, 'y4':y4}



n_exp = 1 : a11, a41
    
n_exp = 2 : a11,a12, a21,a22, a31, a41,a42
    
n_exp = 3 : a11,a12,a13, a21,a22,a23, a31,a32, a41,a42
    
n_exp = 4 : a11,a12,a13,a14, a21,a22,a23,a24, a31,a32,a33, a41,a42,a43,a44


### Make parameter dict

In [6]:
def f_make_pars(num_exp,use_prior):
    
    par_dict={}
    par_dict = gv.BufferDict()         # any dictionary works
    
    if num_exp==1:
        vals_a=[1,1e-4]; vals_e=[1]
    elif num_exp==2: 
        vals_a=[1,1e4,1e-4,1,1e-4,1e-4,1e-4]; vals_e=[1,2,3]
    elif num_exp==3:
        vals_a=[1,1,1,1e-4,1,1,1e-4,1e-4,1e-4,1e-4]; vals_e=[1,2,3,5]
    elif num_exp==4:
        vals_a=[1,1,1,1, 1e-4,1,1,1, 1e-4,1e-4,1, 1e-4,1e-4,1e-4,1]; vals_e=[1,2,3,5,6,7]
    
    
    e_par_list=[['E1'],['E1','E2','E3'],['E1','E2','E3','E5'],['E1','E2','E3','E5','E6','E7']]
    keys_e=e_par_list[num_exp-1] # Pick element of above list
    
    a_par_list=[['a11','a41'],['a11','a12','a21','a22','a31','a41','a42'],
                ['a11','a12','a13','a21','a22','a23','a31','a32','a41','a42'],
                ['a11','a12','a13','a14','a21','a22','a23','a24','a31','a32','a33', 'a41','a42','a43','a44'] ]
    keys_a=a_par_list[num_exp-1] # Pick element of above list
    
    num_corrs=4
    
    
    w_factor = 0.3 # fraction of uncertainty for prior
    
    for count,key in enumerate(keys_e):
        if use_prior: 
            par_dict[key]=gv.gvar(vals_e[count],vals_e[count]*w_factor)
        else : 
            par_dict[key]=vals_e[count]
    
    for count,key in enumerate(keys_a):
        if use_prior: 
            par_dict[key]=gv.gvar(vals_a[count],vals_a[count]*w_factor)
        else : 
            par_dict[key]=vals_a[count]

    par_dict['const1']=1
#     par_dict['const2']=1
#     par_dict['const3']=1
    par_dict['const4']=1
    
    return par_dict

# f_make_pars(3,True)

## Load data

In [7]:
dict_global={}
data_dict={}
dict_global['4']=  {'2pt':1.126500594310e-02, 'c':2.758810226094e-01}
dict_global['8']=  {'2pt':5.591911340388e-03, 'c':1.395714549742e-01}
dict_global['16']= {'2pt':2.790831203460e-03, 'c':6.999296068404e-02}
dict_global

{'4': {'2pt': 0.0112650059431, 'c': 0.2758810226094},
 '8': {'2pt': 0.005591911340388, 'c': 0.1395714549742},
 '16': {'2pt': 0.00279083120346, 'c': 0.06999296068404}}

In [8]:
data_dir='../data/free_theory/'

s_list=[4,8,16]
for s in s_list:
    print(data_dir+'s2xr_free_q5k{0}t*_4pt_pl.dat'.format(s))
    fname=glob.glob(data_dir+'s2xr_free_q5k{0}t*_4pt_pl.dat'.format(s))[0]
    print(fname,s)
    df,Lt=f_get_data_df(fname)
    data_dict[str(s)]={'df':df,'Lt':Lt}

../data/free_theory/s2xr_free_q5k4t*_4pt_pl.dat
../data/free_theory/s2xr_free_q5k4t64_4pt_pl.dat 4
4 64
../data/free_theory/s2xr_free_q5k8t*_4pt_pl.dat
../data/free_theory/s2xr_free_q5k8t128_4pt_pl.dat 8
8 128
../data/free_theory/s2xr_free_q5k16t*_4pt_pl.dat
../data/free_theory/s2xr_free_q5k16t256_4pt_pl.dat 16
16 256


$$ \frac{\langle \phi_4 \rangle }{ \langle \phi_2^2 \rangle} = \sum_{l,\Delta} A_{l,\Delta} \cosh\left[c \ (\Delta + l ) (t-T/2) \right] $$

### Create object

In [9]:
s=16
df_temp=data_dict[str(s)]['df']
df1=df_temp[df_temp.l==0]
df2=df_temp[df_temp.l==2]
df3=df_temp[df_temp.l==4]
df4=df_temp[df_temp.l==6]

Corr=corr(df1=df1,df2=df2,df3=df3,df4=df4,c=dict_global[str(s)]['c'],s=s,Lt=data_dict[str(s)]['Lt'])


### Interactive Fit

In [10]:
tmax=int(np.max(df_temp.t.values)) # Max value of t obtained from dataframe

func_w=Dropdown(options=[f_multi_exp2],value=f_multi_exp2,description='fit func',disabled=False)
tr1=IntRangeSlider(value=[20,tmax],min=0,max=tmax,step=1,description='trange1')
tr2=IntRangeSlider(value=[20,tmax],min=0,max=tmax,step=1,description='trange2')
tr3=IntRangeSlider(value=[20,tmax],min=0,max=tmax,step=1,description='trange3')
tr4=IntRangeSlider(value=[20,tmax],min=0,max=tmax,step=1,description='trange4')
pr=IntRangeSlider(value=[0,tmax],min=0,max=tmax,step=1,description='plot range')
pr_w=widgets.Checkbox(value=False,description='use_prior',indent=False,disabled=False)
plt_w=Checkbox(value=True,description='plot',indent=False,disabled=False)
numexp_w=IntSlider(value=2,min=1,max=8,step=1,description='num_exp')
verbose_w=IntSlider(value=1,min=-1,max=2,step=1,description='verbose_w')

# (self,f_make_pars,func,fit_range,plt_range,num_exp=3,verbose=0,use_prior=False,plot=True)

v1=VBox([tr1,tr2])
v2=VBox([tr3,tr4])
v3=VBox([numexp_w,verbose_w])
v4=VBox([pr_w,plt_w])
v5=VBox([pr,func_w])

ui=HBox(children=[v1,v2,v3,v4,v5])
        
out=interactive_output(Corr.f_perform_fit_exp, {'f_make_pars':fixed(f_make_pars),
         'func':func_w,'fit_range1':tr1,'fit_range2':tr2,'fit_range3':tr3,'fit_range4':tr4, 'plt_range':pr,'num_exp':numexp_w, 
         'verbose':verbose_w,'use_prior':pr_w,'plot':plt_w})

display(out,ui)

Output()